In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

# Base folder inside Google Drive
BASE_FOLDER = "/content/drive/MyDrive/P1_VendorPerformance"

# Paths for data and logs
DATA_FOLDER = os.path.join(BASE_FOLDER, "data")
LOG_FOLDER = os.path.join(BASE_FOLDER, "logs")

# Make sure logs folder exists
os.makedirs(LOG_FOLDER, exist_ok=True)

# Logging configuration
logging.basicConfig(
    filename=os.path.join(LOG_FOLDER, "ingestion_db.log"),
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

# Save inventory.db inside Google Drive
DB_PATH = "/content/drive/MyDrive/P1_VendorPerformance/inventory.db"
engine = create_engine(f'sqlite:///{DB_PATH}')

def ingest_db_in_chunks(file_path, table_name, engine, chunksize=100000):
    '''Read CSV in chunks and ingest into DB without running out of memory'''
    try:
        for chunk in pd.read_csv(file_path, engine='python', chunksize=chunksize):
            chunk.to_sql(table_name, con=engine, if_exists='append', index=False)
        logging.info(f"✅ Successfully ingested {file_path} in chunks")
    except Exception as e:
        logging.error(f"❌ Failed to ingest {file_path}: {e}")
        print(f"⚠️ Skipping {file_path} due to error: {e}")

def load_raw_data():
    '''This function will load the CSVs as dataframe and ingest into db'''
    start = time.time()
    for file in os.listdir(DATA_FOLDER):
        if file.endswith('.csv'):
            file_path = os.path.join(DATA_FOLDER, file)
            logging.info(f'Ingesting {file} in db')
            ingest_db_in_chunks(file_path, file[:-4], engine)
    end = time.time()
    total_time = (end - start) / 60
    logging.info('--------------- Ingestion Complete ---------------')
    logging.info(f'Total Time Taken: {total_time:.2f} minutes')

if __name__ == '__main__':
    load_raw_data()


In [8]:
for file in os.listdir('/content/drive/MyDrive/P1_VendorPerformance/data'):
    if '.csv' in file:
        df = pd.read_csv('/content/drive/MyDrive/P1_VendorPerformance/data/' + file)
        print(df.shape)

(224489, 9)
(206529, 9)
(12261, 9)
(2372474, 16)
(5543, 10)
(12825363, 14)


In [6]:
import sqlite3
import pandas as pd
# Connect to the database
conn = sqlite3.connect('inventory.db')

# Get list of tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables

,name
0,end_inventory
1,begin_inventory
2,purchase_prices
3,purchases
4,vendor_invoice
5,sales
